# Hamming weight and Pearson Correlation

This notebook demonstrates how to guess a floating number using Hamming Weight model.


We generate a random secret floating number. Then we use hamming weight model to guess the number.

We develop a method of devide-and-conquer to quickly arrive to the close number with a defined precision.

## Guessing...

In [1]:
import struct
import numpy as np
import pandas as pd
import random

from guess_range import *

## Test single point

In [2]:
#
# initalize parameter and generate a random number (secret number)
number_tests = 1000
known_inputs = np.random.uniform(-1e0, 1e0, 2000) * 1e1
guess_range = (-1e0, 1.0e0)
secret_number = random.uniform(guess_range[0], guess_range[1])
prescision = 1e-6

In [ ]:
guess_range = guess_number_range(secret_number, guess_range, prescision, number_tests, known_inputs)

***guess_range = (-1.0, 1.0) (0.000000)
low_corr = 0.716625127762014 high_corr = 0.6785802214537688
***guess_range = (-1.0, 0.0) (-0.500000)
low_corr = 0.3406623410053239 high_corr = 0.7176248696640564
***guess_range = (-0.5, 0.0) (-0.250000)
low_corr = 0.7282975995943664 high_corr = 0.2506772555643455
***guess_range = (-0.5, -0.25) (-0.375000)
low_corr = 0.6074492825898015 high_corr = 0.7589381115488711
***guess_range = (-0.375, -0.25) (-0.312500)
low_corr = 0.7731555296313124 high_corr = 0.4215550562476784
***guess_range = (-0.375, -0.3125) (-0.343750)
low_corr = 0.8004039315648505 high_corr = 0.535725381339844
***guess_range = (-0.375, -0.34375) (-0.359375)
low_corr = 0.8411773270029455 high_corr = 0.6092706131848464
***guess_range = (-0.375, -0.359375) (-0.367188)
low_corr = 0.8411773270029455 high_corr = 0.6605887205894699
***guess_range = (-0.375, -0.3671875) (-0.371094)
low_corr = 0.6824818941645354 high_corr = 0.8411773270029455
***guess_range = (-0.37109375, -0.3671875) (-0.36

In [ ]:
guessed_number = (guess_range[1] + guess_range[0]) / 2.0
print('the secret number =', secret_number)
print('the guessed number =', guessed_number)
error_rate = abs((guessed_number - secret_number) / secret_number) * 100
print('error rate = %0.4f' % error_rate, '%')

## Test multiple points

In [ ]:
# initialize
guess_range = (-3e0, 3e0)
prescision = 1e-4
number_tests = 100

n_secret_numbers = 50
secret_numbers = np.concatenate((
    np.random.uniform(guess_range[0], guess_range[1], int(n_secret_numbers*2/4) ),
    np.random.uniform(guess_range[0], guess_range[1], int(n_secret_numbers*1/4)) * 3e-1,
    np.random.uniform(guess_range[0], guess_range[1], int(n_secret_numbers*1/4)) * 1e-1
))

known_inputs = np.random.uniform(-1.0, 1.0, 2000) * 1e1

print('secret_numbers.shape = %s' % (str(secret_numbers.shape)))
print('known_inputs.shape = %s' % (str(known_inputs.shape)))

In [ ]:
results = pd.Series(index=secret_numbers, name='guessed_numbers', dtype=np.float32)
for idx in results.index:
    grange = guess_number_range(idx, guess_range, prescision, number_tests, known_inputs)
    results[idx] = (grange[1] + grange[0]) / 2.0
    print('secret_value = %f, guessed_value = %f' % (idx, results[idx]))
results.sort_index(inplace=True)

In [ ]:
ax = results.plot(figsize = (12, 6), marker='.')
ax.plot(results.index, results.index, marker='.', linewidth=1, label='secret_numbers')
ax.legend()
ax.set_xlabel('secret values')
ax.set_ylabel('guessed values')
ax.grid(True)

In [ ]:
error_rate = ((results - results.index)/results.index).abs()
ax = error_rate.plot(figsize = (12, 6), label='error rate')
ax.legend()
ax.set_xlabel('secret values')
ax.set_ylabel('error rates')
ax.grid(True)

# Batinna method

In [ ]:
def batina_guess_number(secret_number, guess_range, prescision, known_inputs):
    low, high = guess_range
    guess_val = np.arange(low, high, prescision)
    hw = pd.DataFrame(columns=guess_val,
                        data=np.vectorize(hamming_weight)(known_inputs.reshape(-1, 1) * guess_val))
    hw['actual'] = np.vectorize(hamming_weight)(known_inputs * secret_number)
    return hw.corr(method='pearson')['actual'].drop('actual').idxmax()

In [ ]:
batinta_results = pd.Series(index=secret_numbers, name='guessed_numbers', dtype=np.float32)
for idx in batinta_results.index:
    # we have to reduce the precision, otherwise, it takes too long time to run
    batinta_results[idx] = batina_guess_number(idx, guess_range, prescision * 100, known_inputs)
    print('secret_value = %f, guessed_value = %f' % (idx, batinta_results[idx]))
batinta_results.sort_index(inplace=True)    

In [ ]:
ax = batinta_results.plot(figsize = (12, 6), marker='.', label='batina')
results.plot(ax=ax, marker='.', label='bxlab')
ax.plot(batinta_results.index, batinta_results.index, marker='.', linewidth=1, label='secret_numbers')
ax.legend()
ax.set_xlabel('secret values')
ax.set_ylabel('guessed values')
ax.grid(True)

In [ ]:
batina_error_rate = ((batinta_results - batinta_results.index)/batinta_results.index).abs()
ax = batina_error_rate.plot(figsize = (12, 6), label='batina error rate')
ax = error_rate.plot(ax=ax, label='error rate')
ax.legend()
ax.set_xlabel('secret values')
ax.set_ylabel('error rates')
ax.grid(True)